<a href="https://colab.research.google.com/github/nazzzus/hs_of_detection/blob/main/hs_of_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#helpful packages
import numpy as np # linear algebra
import pandas as pd # data processing, csv file I/O (bsp. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#for data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
dataset = pd.read_csv('/content/labeled_data.csv')
dataset.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [12]:
dt_transformed = dataset[['class', 'tweet']]
y = dt_transformed.iloc[:,:-1].values

In [13]:
#Encoding the dependent variable

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
y = np.array(ct.fit_transform(y))

In [14]:
print(y)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [15]:
y_df = pd.DataFrame(y)
y_hate = np.array(y_df[0])
y_offensive = np.array(y_df[1])

In [16]:
print(y_hate)
print(y_offensive)

[0. 0. 0. ... 0. 0. 0.]
[0. 1. 1. ... 1. 1. 0.]


In [18]:
#Cleaning the Texts

corpus = []
for i in range(0, 24783):
    review = re.sub('[^a-zA-Z]', ' ', dt_transformed['tweet'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()

In [23]:
#split dataset in 2 sets - training set & data set

X_train, X_test, y_train, y_test = train_test_split(X, y_hate, test_size = 0.30, random_state = 0)

In [24]:
#finding best model to predict hate speech

#Naive Bayes

classifier_np = GaussianNB()
classifier_np.fit(X_train, y_train)

GaussianNB()

In [25]:
#finding best model to predict hate speech

#Logistic Regression
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [26]:
#finding best model to predict hate speech

#SVM Classifier

classifier_svm = svm.SVC()
classifier_svm.fit(X_train, y_train)

SVC()

In [27]:
#making confusion matrix for each model

#Naive Bayes

y_pred_np = classifier_np.predict(X_test)
cm = confusion_matrix(y_test, y_pred_np)
print(cm)

[[3289 3719]
 [ 168  259]]


In [28]:
#making confusion matrix for each model

#SVM

y_pred_svm = classifier_svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(cm)

[[6964   44]
 [ 372   55]]


In [29]:
#making confusion matrix for each model

#logistic regression

y_pred_lr = classifier_lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(cm)

[[6911   97]
 [ 347   80]]


In [31]:
#show accuracy of each model

np_score = accuracy_score(y_test, y_pred_np)
svm_score = accuracy_score(y_test, y_pred_svm)
lr_score = accuracy_score(y_test, y_pred_lr)

print('Naive Bayes Accuracy: ', str(np_score))
print('Support Vector Machine Accuracy: ', str(svm_score))
print('Logistic Regression Accuracy', str(lr_score))

Naive Bayes Accuracy:  0.4772024209818426
Support Vector Machine Accuracy:  0.9440484196368527
Logistic Regression Accuracy 0.9402824478816408
